# CLI

> This is the demo_garden cli

In [3]:
#| default_exp cli

In [21]:
#| export
from fastcore.script import *
import os
from pathlib import Path
from fh_bootstrap import *

In [6]:
#| hide
from nbdev.showdoc import *


In [25]:
#| export

def generate_sample_requirements_txt():
    """Generates sample requirements.txt"""
    return """
python-fasthtml

    """

def generate_sample_readme_content():
    """Generates boiler plate README.md content"""
    return """
This is a sample demo using fasthtml. 

To run:

`pip install -r requirements.txt`

`python app.py`

    """

def generate_sample_fasthtml_content():
    """Generates boiler plate fasthtml content"""
    return """
# This was autogenerated as a sample application! 
from fh_bootstrap import *
app, rt = fast_app(pico=False, live=False)
        
@rt("/")
def get():
    return Titled(
            "Sample Demo",
            H1("This is a sample demo!")
            )
serve() 
"""

def generate_sample_dockerfile_content():
    """Generate boilerplate Dockerfile content."""
    return """
# Use an official Python runtime as a parent image
FROM python:3.9-slim

# Set the working directory in the container
WORKDIR /app

# Copy the current directory contents into the container at /app
COPY . /app

# Install any needed packages specified in requirements.txt
RUN pip install --no-cache-dir -r requirements.txt

# Make port 8000 available to the world outside this container
EXPOSE 8000

# Define environment variable
ENV NAME World

# Run app.py when the container launches
CMD ["python", "app.py"]
"""

def generate_sample_cloudbuild_yaml_content():
    """Generate boilerplate cloudbuild.yaml content."""
    return """
steps:
# Build the container image
- name: 'gcr.io/cloud-builders/docker'
  args: ['build', '-t', 'gcr.io/$PROJECT_ID/$REPO_NAME:$COMMIT_SHA', '.']
# Push the container image to Container Registry
- name: 'gcr.io/cloud-builders/docker'
  args: ['push', 'gcr.io/$PROJECT_ID/$REPO_NAME:$COMMIT_SHA']
# Deploy container image to Cloud Run
- name: 'gcr.io/google.com/cloudsdktool/cloud-sdk'
  entrypoint: gcloud
  args:
  - 'run'
  - 'deploy'
  - '$REPO_NAME'
  - '--image'
  - 'gcr.io/$PROJECT_ID/$REPO_NAME:$COMMIT_SHA'
  - '--region'
  - 'us-central1'
images:
- 'gcr.io/$PROJECT_ID/$REPO_NAME:$COMMIT_SHA'
"""

In [26]:
#| export
@call_parse
def demo_garden_init(project_name: str = None, # this is the path for the directory you want demo garden to manage 
                    ):
    """Initializes a new Demo Garden project. Call this function when inside of the directory
    you want Demo Garden to manage. Using this like `git init`.
    """
    if project_name:
        project_dir = Path(project_name)
    else:
        project_dir = Path.cwd()
    if (project_dir / ".demogarden").exists():
        raise ValueError(f"Error: This is already a demogarden managed repository")
    if project_name and project_dir.exists():
        raise ValueError(f"Error: {project_name} already exists")

    (project_dir / "assets").mkdir(exist_ok=True)
    (project_dir / "demos").mkdir(exist_ok=True)
    (project_dir / "demos" / "Sample_Category" / "Demo_Name").mkdir(parents=True, exist_ok=True)
    with open(project_dir / "demos" / "Sample_Category" / "Demo_name" / "app.py", "w") as f:
        f.write(generate_sample_fasthtml_content())
    with open(project_dir / "demos" / "Sample_Category" / "Demo_Name" / "requirements.txt", "w") as f:
        f.write(generate_sample_requirements_txt())
    with open(project_dir / "demos" / "Sample_Category" / "Demo_Name" / "README.md", "w") as f:
        f.write(generate_sample_readme_content())

    # Generate Dockerfile for Demo Garden System. This is something that won't be configured by the user
    with open(project_dir / "Dockerfile", "w") as f:
        f.write(generate_sample_dockerfile_content())
    with open(project_dir / "cloudbuild.yaml", "w") as f:
        f.write(generate_sample_cloudbuild_yaml_content())
        
    with open(project_dir / "demos" / "Sample_Category" / "Demo_Name" / "Dockerfile", "w") as f:
        f.write(generate_sample_dockerfile_content())

    with open(project_dir / "demos" / "Sample_Category" / "Demo_Name" / "cloudbuild.yaml", "w") as f:
        f.write(generate_sample_cloudbuild_yaml_content())

    (project_dir / "app.py").touch()
    (project_dir / ".demogarden").touch()
    
    print(f"Demo Garden project initialized successfully in {project_dir}")

In [ ]:
#| export
@call_parse
def demo_garden_build():
    """Build the Demo Garden project."""
    if not Path(".demogarden").exists():
        raise ValueError("Error: Not a demo garden project. Run 'demo_garden_init' to initialize this repository ")
        

In [38]:
from fh_bootstrap import *

In [43]:
show(Navbar("nav", 1, [("About", "/")]))

In [46]:
ContainerT??

Init signature: ContainerT(*values)
Docstring:     
Create a collection of name/value pairs.

Example enumeration:

>>> class Color(Enum):
...     RED = 1
...     BLUE = 2
...     GREEN = 3

Access them by:

- attribute access:

  >>> Color.RED
  <Color.RED: 1>

- value lookup:

  >>> Color(1)
  <Color.RED: 1>

- name lookup:

  >>> Color['RED']
  <Color.RED: 1>

Enumerations can be iterated over, and know how many members they have:

>>> len(Color)
3

>>> list(Color)
[<Color.RED: 1>, <Color.BLUE: 2>, <Color.GREEN: 3>]

Methods can be added to enumerations, and members can have their own
attributes -- see the documentation for details.
Source:        
class ContainerT(BSEnum):
    Basic = ''
    Fluid = 'fluid'
    Sm = 'sm'
    Md = 'md'
    Lg = 'lg'
    Xl = 'xl'
    Xxl = 'xxl'
File:           ~/miniconda3/envs/dev/lib/python3.12/site-packages/fh_bootstrap/__init__.py
Type:           EnumType
Subclasses:     

In [12]:
#| hide
import nbdev; nbdev.nbdev_export()